<a href="https://colab.research.google.com/github/Anshlulla/Exploring-Langchain-and-RAG/blob/main/PromptTemplates_LLMChains_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Caching LLM Responses

###InMemoryCaching

In [1]:
!pip install google-generativeai langchain-google-genai --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 666.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 7.0 MB/s eta 0:00:00


In [2]:
!pip install -U langchain-community --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.2 MB/s eta 0:00:00


In [3]:
from langchain.globals import set_llm_cache
from langchain_google_genai import GoogleGenerativeAI
import os
from getpass import getpass

api_key = getpass()

··········


In [4]:
llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.1, max_output_tokens=256, api_key=api_key)
llm

GoogleGenerativeAI(model='gemini-pro', google_api_key=SecretStr('**********'), temperature=0.1, max_output_tokens=256, client=genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
))

In [5]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = "Tell me a political joke?"
llm.invoke(prompt)

CPU times: user 565 ms, sys: 37.2 ms, total: 602 ms
Wall time: 1.98 s


'Why did the politician cross the road?\n\nTo get to the other side of the aisle.'

In [6]:
%%time
llm.invoke(prompt)

CPU times: user 516 µs, sys: 0 ns, total: 516 µs
Wall time: 523 µs


'Why did the politician cross the road?\n\nTo get to the other side of the aisle.'

###SQLiteCaching

In [7]:
%%time
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))
llm.invoke("Tell me a joke")

CPU times: user 47 ms, sys: 6.03 ms, total: 53 ms
Wall time: 1.12 s


"What do you call a boomerang that won't come back?\n\nA stick."

In [8]:
%%time
llm.invoke("Tell me a joke")

CPU times: user 170 ms, sys: 57 ms, total: 227 ms
Wall time: 494 ms


"What do you call a boomerang that won't come back?\n\nA stick."

##LLM Streaming

In [9]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.1, streaming=True, api_key=api_key)

In [10]:
for chunk in llm.stream("Write a movie script"):
  print(chunk, end="", flush=True)

**INT. APARTMENT - NIGHT**

A young woman, ANNA, 25, sits on the couch, staring at the TV. She is clearly upset.

ANNA
(to herself)
I can't believe he's gone.

She gets up and walks over to the window. She looks out at the city below.

ANNA (cont'd)
I thought we had something special.

She turns away from the window and walks back to the couch. She picks up a picture of her and her boyfriend, MARK.

ANNA (cont'd)
I miss you so much.

She puts the picture down and lies back on the couch. She closes her eyes and tries to sleep.

**INT. APARTMENT - DAY**

Anna wakes up and gets out of bed. She walks over to the window and looks out. The sun is shining and the birds are singing.

ANNA
(to herself)
It's a new day.

She walks over to the bathroom and brushes her teeth. She then goes into the kitchen and makes a cup of coffee.

ANNA (cont'd)
I'm going to be okay.

She takes a sip of her coffee and smiles.

**INT. PARK - DAY**

Anna is sitting on a bench in the park. She is reading a book.

A 

##PromptTemplates

> Useful in scenarios of Question Answering or Completing Sentences

In [11]:
from langchain.prompts import PromptTemplate

template = "You are an experienced SongWriter. Write a short song about {topic} in {language}"
prompt = PromptTemplate.from_template(template=template)
prompt = prompt.format(topic="The Beauty of Mumbai", language="english")
prompt

'You are an experienced SongWriter. Write a short song about The Beauty of Mumbai in english'

In [12]:
output = llm.invoke(prompt)
print(output)

(Verse 1)
In the heart of India's west,
Where dreams and chaos intertwine,
Lies a city of vibrant zest,
Mumbai, a beauty so divine.

(Chorus)
Oh, Mumbai, Mumbai,
A symphony of colors and sound,
Where skyscrapers touch the sky,
And ancient streets abound.

(Verse 2)
From the Gateway of India's grace,
To the bustling markets' lively pace,
Each corner tells a different tale,
Of a city that never fails.

(Chorus)
Oh, Mumbai, Mumbai,
A symphony of colors and sound,
Where skyscrapers touch the sky,
And ancient streets abound.

(Bridge)
The sea breeze whispers secrets sweet,
As the sun sets, casting a golden hue,
The city transforms, a vibrant beat,
Where dreams and aspirations come true.

(Verse 3)
From Bollywood's glitz to art's embrace,
Mumbai's culture knows no bounds,
A melting pot of every race,
Where creativity forever resounds.

(Chorus)
Oh, Mumbai, Mumbai,
A symphony of colors and sound,
Where skyscrapers touch the sky,
And ancient streets abound.

(Outro)
In the heart of India's wes

##ChatPromptTemplates

> Useful in scenarios when user wishes to interact with the LLM in a conversation

> HumanMessagePromptTemplate allows you to create dynamic prompts from the User Message

> They help improve usability, scalability and accuracy.

In [13]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage("You respond only in the JSON Format."),
    HumanMessagePromptTemplate.from_template("Top {n} countries in {area} by population.")
])

chat_prompt_template = chat_prompt_template.format(n=10, area="Asia")
chat_prompt_template

'System: You respond only in the JSON Format.\nHuman: Top 10 countries in Asia by population.'

In [14]:
print(llm.invoke(chat_prompt_template))

```json
{
  "countries": [
    {
      "name": "China",
      "population": 1444015000
    },
    {
      "name": "India",
      "population": 1380004385
    },
    {
      "name": "Indonesia",
      "population": 273523615
    },
    {
      "name": "Pakistan",
      "population": 220892340
    },
    {
      "name": "Bangladesh",
      "population": 164689383
    },
    {
      "name": "Japan",
      "population": 126476461
    },
    {
      "name": "Philippines",
      "population": 109581078
    },
    {
      "name": "Vietnam",
      "population": 97338583
    },
    {
      "name": "Turkey",
      "population": 84339067
    },
    {
      "name": "Iran",
      "population": 83992949
    }
  ]
}
```


##Simple Chains

In [15]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.1, api_key=api_key)
template = "You are an experienced ScriptWriter. Write a movie script about {topic}."
prompt = PromptTemplate.from_template(template=template)

# Creating the chain
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

# Returns a dictionary
output = chain.invoke({"topic": "The Beauty of Mumbai"})
print(output["text"])



> Entering new LLMChain chain...
Prompt after formatting:
You are an experienced ScriptWriter. Write a movie script about The Beauty of Mumbai.


<ipython-input-15-7bf7c43cd696>:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt, verbose=True)



> Finished chain.
**INT. MUMBAI - DAY**

A montage of vibrant images of Mumbai:

* The Gateway of India, with boats bobbing in the harbor
* The bustling streets of Colaba Causeway
* The iconic Victoria Terminus railway station
* The crowded beaches of Juhu
* The towering skyscrapers of Bandra Kurla Complex

**NARRATOR (V.O.)**

Mumbai, a city of dreams, a city of contrasts. A city where the ancient and the modern collide, where the sacred and the profane intertwine.

**EXT. MARINE DRIVE - DAY**

A young woman, ANJALI, walks along the promenade, her eyes taking in the beauty of the city.

**ANJALI**

(to herself)

I've always loved Mumbai. It's a city that's always changing, always evolving.

**EXT. CHOWPATTY BEACH - DAY**

A group of children play in the sand, while their parents watch from the sidelines.

**NARRATOR (V.O.)**

Mumbai is a city of people. People from all walks of life, from all corners of the world.

**EXT. DHARAVI SLUM - DAY**

A maze of narrow streets and makeshift h

##SequentialChains

In [16]:
!pip install --upgrade google-generativeai --q
!pip install --upgrade langchain-google-genai --q

In [26]:
!pip install --upgrade google-generativeai langchain --q

In [18]:
!pip install langchain-google-genai --q

In [19]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Initialize the LLM (Gemini) for script generation
llm1 = GoogleGenerativeAI(model="gemini-pro", temperature=0.8, api_key=api_key)
script_prompt = PromptTemplate(
    template="You are an experienced ScriptWriter. Write a detailed movie script about the topic: {topic}.",
    input_variables=["topic"]
)
script_chain = LLMChain(llm=llm1, prompt=script_prompt, output_key="script")

# Initialize the LLM (Gemini) for character descriptions
llm2 = GoogleGenerativeAI(model="gemini-pro", temperature=0.7, api_key=api_key)
character_prompt = PromptTemplate(
    template="Based on the following movie script, provide character descriptions:\n\nScript:\n{script}",
    input_variables=["script"]
)
character_chain = LLMChain(llm=llm2, prompt=character_prompt, output_key="character_descriptions")

# Set up a SimpleSequentialChain to link both chains
overall_chain = SimpleSequentialChain(
    chains=[script_chain, character_chain],
    input_key="topic",
    output_key="character_descriptions",
    verbose=True
)

# Running the chain with the topic as input
output = overall_chain({"topic": "A romantic scene in NYC"})
print(output)


<ipython-input-19-c4276f3074ff>:30: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = overall_chain({"topic": "A romantic scene in NYC"})




> Entering new SimpleSequentialChain chain...

No script was provided, so I cannot provide character descriptions.

> Finished chain.
{'topic': 'A romantic scene in NYC', 'character_descriptions': 'No script was provided, so I cannot provide character descriptions.'}


In [20]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Initialize the LLM (Gemini) for script generation
llm1 = GoogleGenerativeAI(model="gemini-pro", temperature=0.8, api_key=api_key)
script_prompt = PromptTemplate(
    template="You are an experienced ScriptWriter. Write a detailed movie script about the topic: {topic}.",
    input_variables=["topic"]
)
script_chain = LLMChain(llm=llm1, prompt=script_prompt, output_key="script")

# Run the first chain to get the movie script
script_output = script_chain.run({"topic": "A romantic scene in NYC"})
print(script_output)

# Initialize the LLM (Gemini) for character descriptions
llm2 = GoogleGenerativeAI(model="gemini-pro", temperature=0.7, api_key=api_key)
character_prompt = PromptTemplate(
    template="Based on the following movie script, provide character descriptions:\n\nScript:\n{script}",
    input_variables=["script"]
)
character_chain = LLMChain(llm=llm2, prompt=character_prompt, output_key="character_descriptions")

# Run the second chain using the script output as input
character_descriptions_output = character_chain.run({"script": script_output})

# Print the character descriptions
print(character_descriptions_output)



No script was provided, so I cannot provide character descriptions.


<ipython-input-20-acf275d1b1f8>:14: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  script_output = script_chain.run({"topic": "A romantic scene in NYC"})


##LangChain Agents

###Python REPL

In [21]:
!pip install langchain_experimental --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 3.7 MB/s eta 0:00:00


In [22]:
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
python_repl.run("print([n for n in range(1,100) if n%13==0])")

'[13, 26, 39, 52, 65, 78, 91]\n'

In [23]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

agent_executor.invoke("5.1 ** 7.3 ")



> Entering new AgentExecutor chain...
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:Final Answer: 146306.05007233328

> Finished chain.


{'input': '5.1 ** 7.3 ', 'output': '146306.05007233328'}

###DuckDuckGo and Wikipedia

In [27]:
!pip install -q duckduckgo-search wikipedia

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.4 MB/s eta 0:00:00


In [32]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
output = search.invoke("WWhat is John Mayer's best song?")
print(output)

The John Mayer song oozes emotion, as Mayer's lyrics are revealed in a confessionalist tone while smooth, clean electric guitar fills swell into empty spaces between vocal lines. "Gravity, stay the hell away from me." One of fans' favorite things about Mayer is his brutally honest lyrical mastery. List of John Mayer songs, ranked from best to worst by the Ranker community. Bringing a touch of the blues to pop music, John Mayer is a guitar virtuoso and a masterful songwriter who dropped out of the Berklee College of Music to begin his career. All of John Mayer's singles are included here, but real fans know there are other awesome songs to vote on other than "Your Body Is a Wonderland ... The song, which peaked at number 71 on the Billboard Hot 100, was published in 2006 as part of John's album "Continuum."It had a big influence on the charts. If you're a fan, you are well aware. Guitar Habits thinks that it is superb in every way.. Electric blues is Mayer's forte, and "Slow Dancing in 

In [33]:
search.name, search.description

('duckduckgo_search',
 'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.')

In [36]:
# For additional information such as links, use duckduckgosearchresults
from langchain.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
output = search.run("What is John Mayer's best song?")
print(output)

snippet: The John Mayer song oozes emotion, as Mayer's lyrics are revealed in a confessionalist tone while smooth, clean electric guitar fills swell into empty spaces between vocal lines. "Gravity, stay the hell away from me." One of fans' favorite things about Mayer is his brutally honest lyrical mastery., title: 12 Best John Mayer Songs From His Stunning Career So Far - Zing Instruments, link: https://zinginstruments.com/best-john-mayer-songs/, snippet: The breathy hit netted Mayer his first career GRAMMY Award, for Best Male Pop Vocal Performance, at the 45th Annual GRAMMY Awards in 2003. ... 1 10 John Mayer Songs That Show His Versatility, From 'Room For Squares' To Dead & Co; 2 14 Must-Hear Albums In November: The Cure, Dolly Parton, Jin Of BTS, Ab-Soul, & More;, title: 10 John Mayer Songs That Show His Versatility, From 'Room For Squares ..., link: https://www.grammy.com/news/john-mayer-songs-discography-dead-and-company-vegas-residency, snippet: List of John Mayer songs, ranked 

In [39]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region="ind", max_results=3, safesearch="moderate")
search = DuckDuckGoSearchResults(api_key=wrapper, source="news")
output = search.run("India")
print(output)

snippet: India, officially the Republic of India, [j] [21] is a country in South Asia.It is the seventh-largest country in the world by area and the most populous country.Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast, it shares land borders with Pakistan to the west; [k] China, Nepal, and Bhutan to the north; and Bangladesh and ..., title: India - Wikipedia, link: https://en.wikipedia.org/wiki/India, snippet: Learn about India, the most populous country in the world, with a rich and diverse cultural heritage. Explore its history, geography, economy, politics, religions, languages, and more from Britannica., title: India | History, Map, Population, Economy, & Facts | Britannica, link: https://www.britannica.com/place/India, snippet: Learn about India, a country in South Asia with a diverse culture, history, and geography. Find out about its capital, flag, languages, religions, and more., title: India - Simple English W

In [40]:
import re

pattern = r'snippet: (.*?), title: (,*?), link: (.*?)\],'
matches = re.findall(pattern, output, re.DOTALL)

for snippet, title, link in matches:
    print(f"Snippet: {snippet}")
    print(f"Title: {title}")
    print(f"Link: {link}")
    print()

In [43]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

api_wrapper = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
output = wiki.invoke({"query": "llama index"})
print(output)

Page: Vector database
Summary: A vector database, vector store or vector search engine is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor algorithms, so that one can search the database with a query vector to retrieve the closest matching database records.
Vectors are mathematical representations of data in a high-dimensional space. In this space, each dimension correspond


###ReAct Agents

In [51]:
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
!pip install langchainhub -q

In [46]:
llm

ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), temperature=0.8, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7efa9c27af20>, default_metadata=())

In [79]:
from langchain.prompts import ChatPromptTemplate
from langchain import hub
from langchain.agents import initialize_agent, create_react_agent, Tool, AgentExecutor
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are a screenwriter who creates original movie scripts based on given themes
    and context from similar scripts. Use the provided context as inspiration to write a new script
    that fulfills the user's description. Avoid using HTML tags like <center>, avoid
    using **, and return a plain, well-structured dialogue (or script).

    If the context is minimal, proceed by using general suspense elements and improvising details
    to create an engaging scene that fits the description.

    Here are similar scripts for inspiration:
    {context}

    Now, create a new script based on this prompt:
    {question}

    {agent_scratchpad}
    """),
])

prompt = hub.pull("rlm/rag-prompt")
prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]), tools=tools, agent_scratchpad="")
print(type(prompt))
print(prompt.input_variables)
print(prompt.partial_variables)

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
['context', 'question']
{'tool_names': 'python_repl, Wikipedia, DuckDuckGo', 'tools': [Tool(name='python_repl', description='Useful for executing python code', func=<bound method BaseTool.run of PythonREPLTool(python_repl=PythonREPL(globals={'__name__': 'langchain_experimental.tools.python.tool', '__doc__': 'A tool for running python code in a REPL.', '__package__': 'langchain_experimental.tools.python', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x7efa9c3ad000>, '__spec__': ModuleSpec(name='langchain_experimental.tools.python.tool', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7efa9c3ad000>, origin='/usr/local/lib/python3.10/dist-packages/langchain_experimental/tools/python/tool.py'), '__file__': '/usr/local/lib/python3.10/dist-packages/langchain_experimental/tools/python/tool.py', '__cached__': '/usr/local/lib/python3.10/dist-packages/langchain_experimental/tools/python/__pycache__/t

In [80]:
# 1. PythonREPL Tool (for executing Python Code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name="python_repl",
    func=python_repl.run,
    description="Useful for executing python code"
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki_tool = Tool(
    name="Wikipedia",
    func=wiki.run,
    description="Useful when you want to search about a query over Wikipedia"
)

# 3. DuckDuckGoSearch Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name="DuckDuckGo",
    func=search.run,
    description="Useful when you want to search about a query over the internet"
)

tools = [python_repl_tool, wiki_tool, duckduckgo_tool]

In [81]:
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

In [82]:
context = """
[Context of similar scripts here, such as snippets or themes.
For instance, you might include dialogue from a detective thriller or a romance drama for inspiration.]
"""

query = "Create a scene for a suspense thriller where a detective confronts the main suspect in a dimly lit alley."

try:
    result = agent_executor({
        "context": context,
        "question": query
    })
    print("Generated Script:\n", result['output'])
except Exception as e:
    print(f"Error during script generation: {e}")




> Entering new AgentExecutor chain...
I do not have enough information to create a scene for a suspense thriller where a detective confronts the main suspect in a dimly lit alley.Invalid Format: Missing 'Action:' after 'Thought:I do not have enough information about the setting of the story to create a scene based on your request.Invalid Format: Missing 'Action:' after 'Thought:I do not have enough information in the provided context to create a scene for a suspense thriller where a detective confronts the main suspect in a dimly lit alley.Invalid Format: Missing 'Action:' after 'Thought:I do not have enough information to create a scene for a suspense thriller where a detective confronts the main suspect in a dimly lit alley.Invalid Format: Missing 'Action:' after 'Thought:I do not have enough information to create a scene for a suspense thriller where a detective confronts the main suspect in a dimly lit alley.Invalid Format: Missing 'Action:' after 'Thought:I do not have enough in